<div style="background-color: #f5f5f5; padding: 20px; border-radius: 10px; border: 1px solid #e0e0e0; text-align: center;">
  
  <img src="https://i.ibb.co/mLyt3rk/SUT-Logo.png" alt="SUT Logo" width="100">
  
  <h1 style="margin:10px 0 0 0; color:#333;">Sharif University of Technology</h1>
  <h2 style="margin:0; color:#555; font-weight:normal;">Faculty of Mathematical Sciences</h2>
  
  <hr style="border: 1px solid #ccc; margin-top:15px; margin-bottom:15px;">
  
  <h3 style="margin:10px 0 5px 0; color:#0056b3;">HW2: VAEs, GANs, and Normalizing Flows</h3>
  <h4 style="margin:10px 0 5px 0; color:#0056b3;">Prepared by: Mahshid Dehghani</h4>
  <br/>
  
  <div style="font-size: 1.1em; color: #333; line-height: 1.6;">
    <strong>Course:</strong> Generative Models<br>
    <strong>Instructor:</strong> Dr. Fatemeh Seyyedsalehi<br>
    <strong>Semester:</strong> Fall 1404-1405
  </div>

</div>

<div style="background-color: #f5f5f5; border: 1px solid #f5f5f5; padding: 20px; border-radius: 10px; margin-top: 25px; margin-bottom: 25px;">
  
  <h3 style="text-align: center; margin-top: 0; color: #0056b3;">Student Information</h3>
  <p style="text-align: center; color: #555; margin-bottom: 20px; margin-top: 0;">
    Please fill in your details below.
  </p>

  <table style="width: 60%; margin-left: auto; margin-right: auto; border: none; font-size: 1.1em; color: #333;">
    <tbody>
      <tr>
        <td style="text-align: right; padding: 8px; width: 35%;"><strong>Student Name:</strong></td>
        <td style="text-align: left; padding: 8px;"><em>Your Name Here</em></td> <!-- UPDATE THIS LINE -->
      </tr>
      <tr>
        <td style="text-align: right; padding: 8px;"><strong>Student ID:</strong></td>
        <td style="text-align: left; padding: 8px;"><em>Your Student ID Here</em></td>  <!-- UPDATE THIS LINE -->
      </tr>
    </tbody>
  </table>
  
</div>

In [ ]:
!pip install numpy
!pip install matplotlib
!pip install torch
!pip install scikit-learn

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.datasets import load_digits
from sklearn import datasets
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

# **HW2-P3: Density Estimation with Flow-Based Models**

In this assignment, we explore **flow-based generative models** for density estimation. We focus on two models:

1. **RealNVP** ([Papamakarios et al., 2017](https://arxiv.org/abs/1605.08803))
2. **Integer Discrete Flow (IDF)** ([Hogeboom et al., 2019](https://arxiv.org/abs/1905.07376))

The goal of this exercise is to:

- Understand the purpose of **invertible transformations** and **coupling layers**.
- Implement forward and inverse passes in RealNVP.
- Compare continuous and discrete flows.
- Train models on a small dataset and generate new samples.

<br/>

---

## **Dataset: Digits**

We use the **Scikit-Learn Digits dataset**, which contains 1,500 images of size 8×8 pixels. Each pixel value is an integer between 0 and 16.  

- Training set: first 1,000 images  
- Validation set: next 350 images  
- Test set: remaining 150 images  

Each image is flattened into a 64-dimensional vector for the flow models.

In [ ]:
class Digits(Dataset):
    """Scikit-Learn Digits dataset."""

    def __init__(self, mode='train', transforms=None):
        digits = load_digits()
        if mode == 'train':
            self.data = digits.data[:1000].astype(np.float32)
        elif mode == 'val':
            self.data = digits.data[1000:1350].astype(np.float32)
        else:
            self.data = digits.data[1350:].astype(np.float32)

        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        if self.transforms:
            sample = self.transforms(sample)
        return sample

## **RealNVP**
RealNVP uses a sequence of **invertible transformations** to map a complex data distribution to a simple base distribution (e.g., Gaussian).

Each flow step consists of:

1. **Coupling layer**
2. **Permutation layer**
3. **Forward and inverse functions**


### **Questions  (`#TODO`)**

- **Q1:** What is the purpose of **permutation layers** in RealNVP?
- **Q2:** What is the purpose of **dequantization**?
- **Task:** Implement the `f()`, `f_inv()`, and `sample()` methods in the RealNVP class.

**Your Answer:**

- **Q1:**
- **Q2:**
- **Task:** Complete the `#TODO` sections below:

In [ ]:
class RealNVP(nn.Module):
    def __init__(self, nets, nett, num_flows, prior, D=2, dequantization=True):
        super(RealNVP, self).__init__()
        self.dequantization = dequantization

        self.prior = prior
        self.t = torch.nn.ModuleList([nett() for _ in range(num_flows)])
        self.s = torch.nn.ModuleList([nets() for _ in range(num_flows)])
        self.num_flows = num_flows

        self.D = D

    def coupling(self, x, index, forward=True):
        # x: Input, either images (for the first transformation) or outputs from the previous transformation
        # index: It determines the index of the transformation
        # forward: Whether it is a pass from x to y (forward=True), or from y to x (forward=False)

        (xa, xb) = torch.chunk(x, 2, 1)

        s = self.s[index](xa)
        t = self.t[index](xa)

        if forward:
            #yb = f^{-1}(x)
            yb = # TODO
        else:
            #xb = f(y)
            yb = # TODO

        return torch.cat((xa, yb), 1), s

    def permute(self, x):
        return x.flip(1)

    def f(self, x):
        """
        Forward pass: x → z, computes:
        - The transformed variable z
        - The log-determinant of Jacobian

        TODO: Implement the flow:
            1. Loop through all flows
            2. Apply coupling layer
            3. Apply permutation
            4. Accumulate log-det Jacobian = -sum(s)
        """

        log_det_J, z = x.new_zeros(x.shape[0]), x
        # TODO: Implement the RealNVP forward pass
        return z, log_det_J

    def f_inv(self, z):
        """
        Inverse pass: z → x

        TODO: Implement the inverse flow:
            1. Loop **backwards**
            2. Undo permutation
            3. Apply coupling layer in inverse mode (forward=False)
        """

        x = z
        # TODO: Implement the inverse RealNVP pass
        return x

    def forward(self, x, reduction='avg'):
        z, log_det_J = self.f(x)
        if reduction == 'sum':
            return -(self.prior.log_prob(z) + log_det_J).sum()
        else:
            return -(self.prior.log_prob(z) + log_det_J).mean()

    def sample(self, batchSize):
        """
        Sampling:
        TODO: Implement sampling:
            1. Sample z ~ prior
            2. Apply inverse flow f^{-1}(z)
            3. Return generated x

        Important: Output should be shaped (batchSize, D)
        """
        pass

In [ ]:
def evaluation(test_loader, name=None, model_best=None, epoch=None):
    # Evaluation
    if model_best is None:
        # Load the best performing model
        model_best = RealNVP(nets, nett, num_flows, prior, D=D, dequantization=True)
        model_best.load_state_dict(torch.load(name + '.model'))

    model_best.eval()
    loss = 0.
    N = 0.
    for indx_batch, test_batch in enumerate(test_loader):
        if hasattr(model_best, 'dequantization'):
            if model_best.dequantization:
                test_batch = test_batch + (1. - torch.rand(test_batch.shape))/2.
        loss_t = model_best.forward(test_batch, reduction='sum')
        loss = loss + loss_t.item()
        N = N + test_batch.shape[0]
    loss = loss / N

    if epoch is None:
        print(f'Final Loss: nll={loss}')
    else:
        print(f'Epoch: {epoch}, val nll={loss}')

    return loss


def samples_real(name, test_loader):
    # REAL-------
    num_x = 4
    num_y = 4
    x = next(iter(test_loader)).detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], (8, 8))
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name+'_real_images.pdf', bbox_inches='tight')
    plt.title("Real Images")
    plt.show()
    plt.close()


def samples_generated(name, data_loader, extra_name=''):
    x = next(iter(data_loader)).detach().numpy()

    # GENERATIONS-------
    model_best = RealNVP(nets, nett, num_flows, prior, D=D, dequantization=True)
    model_best.load_state_dict(torch.load(name + '.model'))
    model_best.eval()

    num_x = 4
    num_y = 4
    x = model_best.sample(num_x * num_y)
    x = x.detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], (8, 8))
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name + '_generated_images' + extra_name + '.pdf', bbox_inches='tight')
    plt.title("Generated Images")
    plt.show()
    plt.close()


def plot_curve(name, nll_val):
    plt.plot(np.arange(len(nll_val)), nll_val, linewidth='3')
    plt.xlabel('epochs')
    plt.ylabel('nll')
    plt.savefig(name + '_nll_val_curve.pdf', bbox_inches='tight')
    plt.title("Validation Loss")
    plt.show()
    plt.close()

In [ ]:
def training(name, max_patience, num_epochs, model, optimizer, training_loader, val_loader):
    nll_val = []
    best_nll = 1000.
    patience = 0

    # Main loop
    for e in range(num_epochs):
        # Training
        model.train()
        for indx_batch, batch in enumerate(training_loader):
            if hasattr(model, 'dequantization'):
                if model.dequantization:
                    batch = batch + (1. - torch.rand(batch.shape))/2.
            loss = model.forward(batch)

            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()

        # Validation
        loss_val = evaluation(val_loader, model_best=model, epoch=e)
        nll_val.append(loss_val)  # Save for plotting

        if e == 0:
            print('saved!')
            torch.save(model.state_dict(), name + '.model')
            best_nll = loss_val
        else:
            if loss_val < best_nll:
                print('saved!')
                torch.save(model.state_dict(), name + '.model')
                best_nll = loss_val
                patience = 0

                samples_generated(name, val_loader, extra_name="_epoch_" + str(e))
            else:
                patience = patience + 1

        if patience > max_patience:
            break

    nll_val = np.asarray(nll_val)

    return nll_val

### **Initialize Dataloaders**

In [ ]:
train_data = Digits(mode='train')
val_data = Digits(mode='val')
test_data = Digits(mode='test')

training_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

result_dir = 'results/'
if not(os.path.exists(result_dir)):
    os.mkdir(result_dir)
name = 'realnvp'

### **Hyperparams**

In [ ]:
D = 64                  # Input dimension
M = 256                 # The number of neurons in scale (s) and translation (t) nets

lr = 1e-3               # Learning rate
num_epochs = 1000       # Max. number of epochs
max_patience = 20       # An early stopping is used, if training doesn't improve for longer than 20 epochs, it is stopped

### **Initialize RealNVP**

In [ ]:
# The number of invertible transformations
num_flows = 8

# Scale (s) network
nets = lambda: nn.Sequential(nn.Linear(D // 2, M), nn.LeakyReLU(),
                             nn.Linear(M, M), nn.LeakyReLU(),
                             nn.Linear(M, D // 2), nn.Tanh())

# Translation (t) network
nett = lambda: nn.Sequential(nn.Linear(D // 2, M), nn.LeakyReLU(),
                             nn.Linear(M, M), nn.LeakyReLU(),
                             nn.Linear(M, D // 2))

# Prior (a.k.a. the base distribution): Gaussian
prior = torch.distributions.MultivariateNormal(torch.zeros(D), torch.eye(D))
# Init RealNVP
model = RealNVP(nets, nett, num_flows, prior, D=D, dequantization=True)

In [ ]:
# Optimizer
optimizer = torch.optim.Adamax([p for p in model.parameters() if p.requires_grad == True], lr=lr)

In [ ]:
# Training procedure
nll_val = training(name=result_dir + name, max_patience=max_patience, num_epochs=num_epochs, model=model, optimizer=optimizer,
                       training_loader=training_loader, val_loader=val_loader)

In [ ]:
test_loss = evaluation(name=result_dir + name, test_loader=test_loader)
f = open(result_dir + name + '_test_loss.txt', "w")
f.write(str(test_loss))
f.close()

samples_real(result_dir + name, test_loader)

samples_generated(result_dir + name, test_loader)

plot_curve(result_dir + name, nll_val)

## **Integer Discrete Flow**

IDF extends flow-based models to **discrete integer data**. Please read the paper and answer below questions.

### **Questions  (`#TODO`)**

**Basics:**

- **Q3:** Write down the change-of-variable formula for both RealNVP and IDF.
- **Q4:** Compare the forward and inverse transformations of IDF to RealNVP.
- **Q5:** What role does the scaling function s(x) play in RealNVP? Why must it be removed in IDF?
- **Q6:** Why do we need a discrete prior for integer data?
- **Q7:** How does the `RoundStraightThrough` class allow gradients to flow through integer rounding?
- **Q8:** Research the effect of the rounding step in IDF on training.  

**(Bonus) Experiment:**

- Add a small random noise (dequantization) to IDF inputs and observe training behavior.  
  **Q9:** Does this move IDF behavior closer to RealNVP?


**Your Answer:**

- **Q3:**
- **Q4:**
- **Q5:**
- **Q6:**
- **Q7:**
- **Q8:**
- **(Bonus) Q9:**

### **Distributions**

In [ ]:
# Chakraborty & Chakravarty, "A new discrete probability distribution with integer support on (−∞, ∞)",
#  Communications in Statistics - Theory and Methods, 45:2, 492-505, DOI: 10.1080/03610926.2013.830743

def log_min_exp(a, b, epsilon=1e-8):
    """
    Source: https://github.com/jornpeters/integer_discrete_flows
    Computes the log of exp(a) - exp(b) in a (more) numerically stable fashion.
    Using:
     log(exp(a) - exp(b))
     c + log(exp(a-c) - exp(b-c))
     a + log(1 - exp(b-a))
    And note that we assume b < a always.
    """
    y = a + torch.log(1 - torch.exp(b - a) + epsilon)

    return y

def log_integer_probability(x, mean, logscale):
    scale = torch.exp(logscale)

    logp = log_min_exp(
        F.logsigmoid((x + 0.5 - mean) / scale),
        F.logsigmoid((x - 0.5 - mean) / scale))

    return logp

In [ ]:
class RoundStraightThrough(torch.autograd.Function):

    def __init__(self):
        super().__init__()

    @staticmethod
    def forward(ctx, input):
        rounded = torch.round(input, out=None)
        return rounded

    @staticmethod
    def backward(ctx, grad_output):
        grad_input = grad_output.clone()
        return grad_input

In [ ]:
class IDF(nn.Module):
    def __init__(self, netts, num_flows, D=2):
        super(IDF, self).__init__()

        if len(netts) == 1:
            self.t = torch.nn.ModuleList([netts[0]() for _ in range(num_flows)])
            self.idf_git = 1

        elif len(netts) == 4:
            self.t_a = torch.nn.ModuleList([netts[0]() for _ in range(num_flows)])
            self.t_b = torch.nn.ModuleList([netts[1]() for _ in range(num_flows)])
            self.t_c = torch.nn.ModuleList([netts[2]() for _ in range(num_flows)])
            self.t_d = torch.nn.ModuleList([netts[3]() for _ in range(num_flows)])
            self.idf_git = 4

        else:
            raise ValueError('You can provide either 1 or 4 translation nets.')

        self.num_flows = num_flows

        self.round = RoundStraightThrough.apply

        self.mean = nn.Parameter(torch.zeros(1, D))
        self.logscale = nn.Parameter(torch.ones(1, D))

        self.D = D

    def coupling(self, x, index, forward=True):

        if self.idf_git == 1:
            (xa, xb) = torch.chunk(x, 2, 1)

            if forward:
                yb = xb + self.round(self.t[index](xa))
            else:
                yb = xb - self.round(self.t[index](xa))

            return torch.cat((xa, yb), 1)

        elif self.idf_git == 4:
            (xa, xb, xc, xd) = torch.chunk(x, 4, 1)

            if forward:
                ya = xa + self.round(self.t_a[index](torch.cat((xb, xc, xd), 1)))
                yb = xb + self.round(self.t_b[index](torch.cat((ya, xc, xd), 1)))
                yc = xc + self.round(self.t_c[index](torch.cat((ya, yb, xd), 1)))
                yd = xd + self.round(self.t_d[index](torch.cat((ya, yb, yc), 1)))
            else:
                yd = xd - self.round(self.t_d[index](torch.cat((xa, xb, xc), 1)))
                yc = xc - self.round(self.t_c[index](torch.cat((xa, xb, yd), 1)))
                yb = xb - self.round(self.t_b[index](torch.cat((xa, yc, yd), 1)))
                ya = xa - self.round(self.t_a[index](torch.cat((yb, yc, yd), 1)))

            return torch.cat((ya, yb, yc, yd), 1)

    def permute(self, x):
        return x.flip(1)

    def f(self, x):
        z = x
        for i in range(self.num_flows):
            z = self.coupling(z, i, forward=True)
            z = self.permute(z)

        return z

    def f_inv(self, z):
        x = z
        for i in reversed(range(self.num_flows)):
            x = self.permute(x)
            x = self.coupling(x, i, forward=False)

        return x

    def forward(self, x, reduction='avg'):
        z = self.f(x)
        if reduction == 'sum':
            return -self.log_prior(z).sum()
        else:
            return -self.log_prior(z).mean()

    def sample(self, batchSize):
        # Sample z:
        z = self.prior_sample(batchSize=batchSize, D=self.D)
        # x = f^-1(z)
        x = self.f_inv(z)
        return x.view(batchSize, 1, self.D)

    def log_prior(self, x):
        log_p = log_integer_probability(x, self.mean, self.logscale)
        return log_p.sum(1)

    def prior_sample(self, batchSize, D=2):
        # Sample from logistic
        y = torch.rand(batchSize, self.D)
        x = torch.exp(self.logscale) * torch.log(y / (1. - y)) + self.mean
        # And then round it to an integer.
        return torch.round(x)

In [ ]:
def evaluation(test_loader, name=None, model_best=None, epoch=None):
    # Evaluation
    if model_best is None:
        # Load the best performing model
        model_best = IDF(netts, num_flows, D=D)

        # Load state_dict into the model
        model_best.load_state_dict(torch.load(name + '.model'))

    model_best.eval()
    loss = 0.
    N = 0.
    for indx_batch, test_batch in enumerate(test_loader):
        loss_t = model_best.forward(test_batch, reduction='sum')
        loss = loss + loss_t.item()
        N = N + test_batch.shape[0]
    loss = loss / N

    if epoch is None:
        print(f'Final Loss: nll={loss}')
    else:
        print(f'Epoch: {epoch}, val nll={loss}')

    return loss


def samples_real(name, test_loader):
    # REAL-------
    num_x = 4
    num_y = 4
    x = next(iter(test_loader)).detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], (8, 8))
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name+'_real_images.pdf', bbox_inches='tight')
    plt.title("Real Images")
    plt.show()
    plt.close()


def samples_generated(name, data_loader, extra_name=''):
    x = next(iter(data_loader)).detach().numpy()

    # GENERATIONS-------
    model_best = IDF(netts, num_flows, D=D)

    # Load state_dict into the model
    model_best.load_state_dict(torch.load(name + '.model'))
    model_best.eval()

    num_x = 4
    num_y = 4
    x = model_best.sample(num_x * num_y)
    x = x.detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], (8, 8))
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name + '_generated_images' + extra_name + '.pdf', bbox_inches='tight')
    plt.title("Generated Images")
    plt.show()
    plt.close()


def plot_curve(name, nll_val):
    plt.plot(np.arange(len(nll_val)), nll_val, linewidth='3')
    plt.xlabel('epochs')
    plt.ylabel('nll')
    plt.savefig(name + '_nll_val_curve.pdf', bbox_inches='tight')
    plt.title("Validation Loss")
    plt.show()
    plt.close()

In [ ]:
def training(name, max_patience, num_epochs, model, optimizer, training_loader, val_loader):
    nll_val = []
    best_nll = 1000.
    patience = 0

    # Main loop
    for e in range(num_epochs):
        # Training
        model.train()
        for indx_batch, batch in enumerate(training_loader):

            loss = model.forward(batch)

            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()

        # Validation
        loss_val = evaluation(val_loader, model_best=model, epoch=e)
        nll_val.append(loss_val)  # Save for plotting

        if e == 0:
            print('saved!')
            torch.save(model.state_dict(), name + '.model')
            best_nll = loss_val
        else:
            if loss_val < best_nll:
                print('saved!')
                torch.save(model.state_dict(), name + '.model')
                best_nll = loss_val
                patience = 0

                samples_generated(name, val_loader, extra_name="_epoch_" + str(e))
            else:
                patience = patience + 1

        if patience > max_patience:
            break

    nll_val = np.asarray(nll_val)

    return nll_val

### **Initialize Dataloaders**

In [ ]:
train_data = Digits(mode='train')
val_data = Digits(mode='val')
test_data = Digits(mode='test')

training_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

result_dir = 'results/'
if not(os.path.exists(result_dir)):
    os.mkdir(result_dir)
name = 'idf'

### **Hyperparams**

In [ ]:
D = 64                  # Input dimension
M = 256                 # The number of neurons in scale (s) and translation (t) nets

lr = 1e-3               # Learning rate
num_epochs = 1000       # Max. number of epochs
max_patience = 20       # An early stopping is used, if training doesn't improve for longer than 20 epochs, it is stopped

### **Initialize IDF**

In [ ]:
# The number of invertible transformations
num_flows = 8

# This variable defines whether we use:
#   1 - the classic coupling layer proposed in (Hogeboom et al., 2019)
#   4 - the general invertible transformation in (Tomczak, 2020) with 4 partitions
idf_git = 4

if idf_git == 1:
    nett = lambda: nn.Sequential(nn.Linear(D // 2, M), nn.LeakyReLU(),
                                     nn.Linear(M, M), nn.LeakyReLU(),
                                     nn.Linear(M, D // 2))
    netts = [nett]

elif idf_git == 4:
    nett_a = lambda: nn.Sequential(nn.Linear(3 * (D // 4), M), nn.LeakyReLU(),
                                       nn.Linear(M, M), nn.LeakyReLU(),
                                       nn.Linear(M, D // 4))

    nett_b = lambda: nn.Sequential(nn.Linear(3 * (D // 4), M), nn.LeakyReLU(),
                                       nn.Linear(M, M), nn.LeakyReLU(),
                                       nn.Linear(M, D // 4))

    nett_c = lambda: nn.Sequential(nn.Linear(3 * (D // 4), M), nn.LeakyReLU(),
                                       nn.Linear(M, M), nn.LeakyReLU(),
                                       nn.Linear(M, D // 4))

    nett_d = lambda: nn.Sequential(nn.Linear(3 * (D // 4), M), nn.LeakyReLU(),
                                       nn.Linear(M, M), nn.LeakyReLU(),
                                       nn.Linear(M, D // 4))

    netts = [nett_a, nett_b, nett_c, nett_d]

# Init IDF
model = IDF(netts, num_flows, D=D)

In [ ]:
# Optimizer
optimizer = torch.optim.Adamax([p for p in model.parameters() if p.requires_grad == True], lr=lr)

In [ ]:
# Training procedure
nll_val = training(name=result_dir + name, max_patience=max_patience, num_epochs=num_epochs, model=model, optimizer=optimizer,
                       training_loader=training_loader, val_loader=val_loader)

In [ ]:
test_loss = evaluation(name=result_dir + name, test_loader=test_loader)
f = open(result_dir + name + '_test_loss.txt', "w")
f.write(str(test_loss))
f.close()

samples_real(result_dir + name, test_loader)

samples_generated(result_dir + name, test_loader)

plot_curve(result_dir + name, nll_val)

## **Bits-Per-Pixel (bpp) Calculation**

Bits-per-pixel (bpp) measures how efficiently a model encodes data. It represents the average number of bits needed to describe each pixel in your dataset. For a dataset of integer-valued samples $x_i \in \mathbb{Z}^D$ and a model that provides exact log-probabilities $\log p_\theta(x_i)$, bpp is computed as:

$$\text{bpp} = - \frac{1}{N \cdot D} \sum_{i=1}^{N} \log_2 p_\theta(x_i)$$

where:  
- $N$ = number of samples  
- $D$ = number of pixels per sample  

**Task:**  

1. Use the trained **RealNVP** and **IDF** model to compute log-probabilities for all samples in the **test set**.  
2. Use these log-probabilities to calculate the **bits-per-pixel (bpp)** for the test set.

In [ ]:
# TODO: bbp calculation

### **Questions (`#TODO`)**

- **Q10:** Compare the quality of generated samples from RealNVP and IDF. What differences do you notice?
- **Q11:** Compare the bpp values obtained from the two models (RealNVP vs IDF). Explain why is suitable for **lossless compression**?

**Your Answer:**

- **Q10:**
- **Q11:**